In [3]:
import pandas as pd
import requests
import json
import time
import os

In [4]:
root_path = '../Dataset'

raw_path = os.path.join(root_path, 'Raw')

if not os.path.exists(raw_path):
    os.makedirs(raw_path)

In [56]:
df = pd.read_csv(os.path.join(raw_path, 'SO.csv'), encoding='ISO-8859-1')
df.head()

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
0,How do I merge SSIS package files?,<p>I am wondering if anyone has any advice on ...,5,0,2011-03-29 05:10:36.293000 UTC,1.0,2011-08-06 03:01:31.267000 UTC,10,version-control|merge|dvcs|ssis,12003,...,"Muncie, IN",950,183,7,125,<p>Using the free Visual Studio add-in <a href...,1.0,2011-08-30 16:29:00.047000 UTC,NaN,8.0
1,Is it good to commit files often if using Merc...,<p>It seems that it is suggested we can commit...,4,0,2010-06-10 18:47:32.023000 UTC,NaN,NaN,3,git|mercurial|dvcs,829,...,NaN,141372,1451,39,12992,<p>The best way to manage this (whether you ar...,4.0,2010-06-10 19:00:34.010000 UTC,NaN,11.0
2,How to drop a column from a Databricks Delta t...,<p>I have recently started discovering Databri...,7,1,2019-01-31 09:15:51.593000 UTC,2.0,2019-05-16 01:28:21.263000 UTC,16,sql|apache-spark|apache-spark-sql|databricks|d...,32123,...,NaN,2591,307,0,549,<p>There is no drop column option on Databrick...,4.0,2019-01-31 10:07:56.103000 UTC,NaN,16.0
3,What is the pyspark equivalent of MERGE INTO f...,"<p>The databricks <a href=""https://docs.databr...",1,2,2020-04-02 07:47:29.583000 UTC,2.0,NaN,5,merge|databricks|delta|delta-lake,7875,...,"Antwerpen, Belgium",655,302,9,89,<p>I managed to find the documentation using t...,2.0,2020-04-05 09:50:08.793000 UTC,NaN,9.0
4,Is a DVCS useful for one developer?,<p>Or would a conventional client-server VCS b...,6,0,2008-10-07 15:53:48.963000 UTC,5.0,2011-12-09 19:03:35.160000 UTC,16,version-control|dvcs,1018,...,"Rochester, NY, United States",112376,2947,204,15424,<p>Since you can still push to another machine...,1.0,2008-10-07 15:56:30.487000 UTC,NaN,15.0


In [57]:
tags = {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench', 'google-cloud-vertex-ai', 'databricks-unity-catalog',
        'databricks-ml', 'amazon-sagemaker', 'amazon-sagemaker-experiments', 'mlflow', 'dvc', 'wandb', 'clearml', 'comet-ml', 'neptune', 'pachyderm', 'lakefs'}

In [58]:
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [59]:
arity = -1
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'{arity}: {post_number}')

0: 4551
1: 1716
2: 75
3: 5


In [78]:
filter_path = os.path.join(root_path, 'Filter')

if not os.path.isdir(filter_path):
    os.mkdir(filter_path)

df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid.to_csv(os.path.join(filter_path, 'All_valid_posts.csv'),
                encoding='ISO-8859-1', index=False)

In [76]:
df_qualified = df_valid[df_valid['Question_score'] > -1]
df_qualified.to_csv(os.path.join(
    filter_path, 'All_qualified_posts.csv'), encoding='ISO-8859-1', index=False)
df_qualified = pd.read_csv(os.path.join(
    filter_path, 'All_qualified_posts.csv'), encoding='ISO-8859-1')
df_qualified.groupby(['Question_valid_tags'])[
    'Question_valid_tags'].agg(['count'])

,count
Question_valid_tags,
['amazon-sagemaker'],764
"['azure-machine-learning-service', 'azure-machine-learning-studio']",11
['azure-machine-learning-service'],232
"['azure-machine-learning-studio', 'azure-machine-learning-service']",35
['azure-machine-learning-studio'],191
"['azure-machine-learning-workbench', 'azure-machine-learning-service']",1
"['azure-machine-learning-workbench', 'azure-machine-learning-studio', 'azure-machine-learning-service']",4
"['azure-machine-learning-workbench', 'azure-machine-learning-studio']",7
['azure-machine-learning-workbench'],3


In [79]:
df_qualified_completed = df_qualified.dropna(subset=['Answer_body'])
df_qualified_completed.to_csv(os.path.join(
    filter_path, 'All_qualified_completed_posts.csv'), encoding='ISO-8859-1', index=False)
df_qualified_completed.groupby(['Question_valid_tags']).agg(['count'])
df_qualified_completed.groupby(['Question_valid_tags'])[
    'Question_valid_tags'].agg(['count'])

,count
Question_valid_tags,
['amazon-sagemaker'],28
['azure-machine-learning-service'],7
"['azure-machine-learning-studio', 'azure-machine-learning-service']",1
['azure-machine-learning-studio'],8
['dvc'],3
['google-cloud-vertex-ai'],1
['mlflow'],8


In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break


with open(os.path.join(raw_path, 'DVC.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)

In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

with open(os.path.join(raw_path, 'Weights & Biases.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)

In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

with open(os.path.join(raw_path, 'Guild AI.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

with open(os.path.join(raw_path, 'SigOpt.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from dateutil import parser
import time
import json


def convert2num(num):
    try:
        return int(num)
    except:
        try:
            return int(num.split()[0])
        except:
            return 0


def get_data(driver, url):
    driver.get(url)

    total_dict = {}

    # question_title
    title = driver.find_element(By.XPATH, '//h1[@class="KPwZRb gKR4Fb"]').text
    # print("question_title:", title)

    # question_view_count
    view_count = driver.find_element(
        By.XPATH, '//div[@class="Nadu4b"]').get_attribute('innerText')
    view_count = convert2num()
    # print("question_view_count:", view_count)

    # sections
    section_lst = driver.find_elements(By.XPATH, '//div[@role="list"]/section')
    # print("list_number:", len(section_lst))

    for i in range(len(section_lst)):
        date = section_lst[i].find_element(
            By.XPATH, './/span[@class="zX2W9c"]').text
        date = parser.parse(date).isoformat()
        body = section_lst[i].find_element(
            By.XPATH, './/div[@class="ptW7te"]').get_attribute("innerText").strip()

        if i < 1:
            total_dict["Question_title"] = title
            total_dict["Question_creation_date"] = date
            total_dict["Question_link"] = url
            total_dict["Question_answer_count"] = len(section_lst) - 1
            total_dict["Question_view_count"] = view_count
            total_dict["Question_body"] = body
            total_dict["Answers"] = []
            # print("question_time:", answer_time)
            # print("question_body:", answer_body)
        else:
            total_dict["Answers"].append({
                "Answer_creation_time": date,
                "Answer_body": body
            })
            # print("answer_time:", answer_time)
            # print("answer_body:", answer_body)

    return total_dict


def get_url(driver):
    urls_lst = driver.find_elements(By.XPATH, '//div[@jscontroller="MAWgde"]')
    for i in range(len(urls_lst)):
        urls_lst[i] = urls_lst[i].find_element(
            By.XPATH, './/a[@class="ZLl54"]').get_attribute('href')
    return urls_lst


In [ ]:
if True:
    driver = webdriver.Chrome(service=ChromeService(
        ChromeDriverManager().install()))

    base_url = 'https://groups.google.com/g/mlflow-users'

    res_dict = []
    page_count = 0

    driver.get(base_url)

    i=0
    next_button = driver.find_element(By.XPATH, '//div[@role="button" and @jslog="95005; track:JIbuQc"]')
    while True:
        
        #is_quit = False
        i+=1
        #if next_button.is_enabled():
        #    next_button.click()
        #else:
        #    break
        
        try:
            next_button.click()
            print(i)
        except:
            break
        
        
        time.sleep(1)
        
        #time.sleep(3)

        #cur_urls_lst = get_url()

        #for cur_url in cur_urls_lst:
        #    if cur_url == 'https://groups.google.com/g/mlflow-users/c/QnASq7ITAoI':
        #        is_quit = True
        #    res_dict.append(get_data(cur_url))

        #if is_quit:
        #    break

        #page_count += 1

    #res_json = json.dumps(res_dict)
    #with open("MLFlow.json", 'w') as f:
    #    f.write(res_json)


In [ ]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import time
import json
import os


def convert2num(num):
    try:
        return int(num)
    except:
        try:
            return int(num.split()[0])
        except:
            return 0


def get_data(driver, url):
    driver.get(url)    
    time.sleep(0.5) 
    
    total_dict = {}

    # question_title
    title = driver.find_element(
        By.XPATH, '//div[@class="QuestionView_title__C_38B"]/h1').text
    # print("Title:", title)

    # question_creation_date
    data_json = driver.find_element(
        By.XPATH, '//script[@id="__NEXT_DATA__"]').get_attribute("innerText")
    data_dict = json.loads(data_json)
    date = data_dict["props"]["pageProps"]["question"]["createdAt"]
    # print("date:", date)

    # question_has_accepted_answer
    try:
        driver.find_element(
            By.XPATH, '//div[@class="AnswerView_acceptedTag__MIxHg"]')
        has_accepted = True
    except:
        has_accepted = False
    # print("has_acceted:", has_accepted)

    # question_view_count
    view_count = driver.find_element(
        By.XPATH, '//div[@class="ant-typography ant-typography-ellipsis ant-typography-single-line ant-typography-ellipsis-single-line Avatar_age___5eSl"]/span').text
    view_count = convert2num(view_count)
    # print("question_view_count:", view_count)

    # question_upvote_count
    upvote_count = driver.find_element(
        By.XPATH, '//div[@class="Vote_textContainer__5bmNJ"]').text
    upvote_count = convert2num(upvote_count)
    # print("question_upvote_count:", upvote_count)

    # question_topics
    topic_lst = driver.find_elements(
        By.XPATH, '//div[@class="Metadata_container__SrVEy"]/div[1]/div[2]/a')
    for i in range(len(topic_lst)):
        topic_lst[i] = topic_lst[i].find_element(By.XPATH, './/span').text
    # print("topics:", topic_lst)

    # qustion_tags
    tag_lst = driver.find_elements(
        By.XPATH, '//div[@class="Metadata_container__SrVEy"]/div[2]/div[2]/a')
    for i in range(len(tag_lst)):
        tag_lst[i] = tag_lst[i].find_element(By.XPATH, './/span').text
    # print("tags:", tag_lst)

    # question_body
    body = driver.find_element(
        By.XPATH, '//div[@class="QuestionView_container__37ZXp"]//div[@class="custom-md-style"]').get_attribute("innerText").strip()
    # print("body:", body)

    # other_answers
    answers_lst = driver.find_elements(
        By.XPATH, '//div[@class="AnswerView_gridContainer__DDxNy"]')
    # print("answer_count:", len(answers_lst))

    answer_count = len(answers_lst)

    total_dict["Questiont_title"] = title
    total_dict["Question_creation_time"] = date
    total_dict["Question_link"] = url
    total_dict["Question_topic"] = topic_lst
    total_dict["Question_tag"] = tag_lst
    total_dict["Question_upvote_count"] = upvote_count
    total_dict["Question_view_count"] = view_count
    total_dict["Question_answer_count"] = answer_count
    total_dict["Question_has_accepted_answer"] = has_accepted
    total_dict["Question_body"] = body

    total_dict["Answers"] = []
    for i in range(len(answers_lst)):
        answer = answers_lst[i]
        # answer_date = answer.find_element(By.XPATH, './/div[@class="ant-typography ant-typography-ellipsis ant-typography-single-line ant-typography-ellipsis-single-line Avatar_age___5eSl"]').text
        answer_date = data_dict["props"]["pageProps"]["question"]["answers"][i]["createdAt"]
        answer_upvote_count = answer.find_element(
            By.XPATH, './/div[@class="Vote_textContainer__5bmNJ"]').text
        answer_upvote_count = convert2num(answer_upvote_count)
        answer_body = answer.find_element(
            By.XPATH, './/div[@class="custom-md-style"]').get_attribute('innerText').strip()

        cur_has_accepted = False
        if has_accepted:
            try:
                answer.find_element(
                    By.XPATH, './/div[@class="AnswerView_acceptedTag__MIxHg"]')
                cur_has_accepted = True
            except:
                cur_has_accepted = False
        else:
            cur_has_accepted = False

        # print("answer_date:", answer_date)
        # print("answer_upvote:", answer_upvote_count)
        # print("anaswer_body:", answer_body)
        # print("answer_has_accepted:", cur_has_accepted)

        total_dict["Answers"].append({
            "Answer_creation_date": answer_date,
            "Answer_upvote_count": answer_upvote_count,
            "Answer_body": answer_body,
            "Answer_has_accepted": cur_has_accepted
        })

    return total_dict


def get_url(driver, url):
    driver.get(url)

    posts_url = []
    urls_lst = driver.find_elements(
        By.XPATH, '//div[@class="QuestionCard_contentContainer__TjFsN QuestionCard_grid__yEHnj"]/a')
    for url_node in urls_lst:
        posts_url.append(url_node.get_attribute('href'))

    next_page_button = driver.find_element(
        By.XPATH, '//li[@title="Next Page"]')
    try:
        next_page_url = next_page_button.find_element(
            By.XPATH, './/a').get_attribute('href')
    except:
        next_page_url = ''

    return posts_url, next_page_url


if True:
    driver = uc.Chrome()

    res_dict = []
    next_page_url = 'https://repost.aws/tags/TAT80swPyVRPKPcA0rsJYPuA/amazon-sage-maker'

    while True:
        cur_urls_lst, next_page_url = get_url(driver, next_page_url)

        for post_url in cur_urls_lst:
            res_dict.append(get_data(driver, post_url))

        if not next_page_url:
            break

    res_json = json.dumps(res_dict)
    with open(os.path.join('../Dataset/Raw', 'Amazon SageMaker.json'), 'w') as f:
        f.write(res_json)

In [78]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import json
import os


def convert2num(num):
    try:
        return int(num)
    except:
        try:
            return int(num.split()[0])
        except:
            return 0


def get_data(driver, url):
    driver.get(url)
    time.sleep(0.5) 

    total_dict = {}

    # question_title
    title = driver.find_element(By.XPATH, '//h2[@class="question-title"]').text
    # print("Title:", title)

    # question_creation_date
    date = driver.find_element(
        By.XPATH, '//span[contains(@class, "action-date")]/time[@role="presentation"]').get_attribute("datetime")
    # print("date:", date)

    # question_has_accepted_answer
    has_accepted = False
    try:
        driver.find_element(By.XPATH, '//span[@aria-label="Best Answer"]')
        has_accepted = True
    except:
        has_accepted = False
    # print("has_acceted:", has_accepted)

    # question_upvote_count
    upvote_count = driver.find_element(
        By.XPATH, '//div[@class="vote-widget"]/span/span[2]').text
    upvote_count = convert2num(upvote_count)
    # print("upvote_count", upvote_count)

    # question_follower_count
    try:
        follower_count = driver.find_element(
            By.XPATH, '//div[@class="widget-content"]/p/strong').text
        follower_count = convert2num(follower_count)
    except:
        follower_count = 0
    # print("question_follower_count", follower_count)

    # question_comment_count
    comment_count = driver.find_element(By.XPATH, '//div[@class="control-bar"]').find_element(
        By.XPATH, '//span[@class="control-score-counter comment-count"]').text
    comment_count = convert2num(comment_count)
    # print("comment_count", comment_count)

    # question_answer_count
    answer_count = driver.find_element(
        By.XPATH, '//span[@itemprop="answerCount"]').text
    # print("answer_count:", answer_count)
    answer_count = convert2num(answer_count)

    # # qustion_tags
    tag_lst = driver.find_elements(
        By.XPATH, '//span[@class="tags"]/a[@class="tag"]')
    for i in range(len(tag_lst)):
        tag_lst[i] = tag_lst[i].find_element(By.XPATH, './a').text
    # print("tags:", tag_lst)

    # question_body
    body = driver.find_element(
        By.XPATH, '//div[@class="question-body post-body"]').get_attribute("innerText").strip()
    # print("body:", body)
    
    total_dict["Question_title"] = title
    total_dict["Question_creation_date"] = date
    total_dict["Question_link"] = url
    total_dict["Question_tag"] = tag_lst
    total_dict["Question_has_accepted_answer"] = has_accepted
    total_dict["Question_answer_count"] = answer_count
    total_dict["Question_comment_count"] = comment_count
    total_dict["Question_follower_count"] = follower_count
    total_dict["Question_upvote_count"] = upvote_count
    total_dict["Question_body"] = body

    total_dict["Answers"] = []

    accepted_answer_lst = driver.find_elements(
        By.XPATH, '//div[@class="post-container answer-container  accepted-answer"]')
    if accepted_answer_lst:
        ac_answer = accepted_answer_lst[0]
        ac_answer_date = ac_answer.find_element(
            By.XPATH, './/time[@role="presentation"]').get_attribute("datetime")
        ac_answer_upvote_count = ac_answer.find_element(
            By.XPATH, './/div[@class="vote-widget"]/span/span[2]').text
        ac_answer_upvote_count = convert2num(ac_answer_upvote_count)
        ac_answer_body = ac_answer.find_element(
            By.XPATH, './/div[@class="answer-body"]').get_attribute('innerText').strip()
        ac_answer_comment_count = ac_answer.find_element(
            By.XPATH, './/span[@class="control-score-counter comment-count"]').get_attribute("innerText")
        ac_answer_comment_count = convert2num(ac_answer_comment_count)

        # print("ac_answer_date:", ac_answer_date)
        # print("ac_answer_upvote:", ac_answer_upvote_count)
        # print("ac_anaswer_body:", ac_answer_body)
        # print("ac_answer_comment_count:", ac_answer_comment_count)
        # print("ac_answer_has_accepted:", True)

        total_dict["Answers"].append({
            "Answer_creation_date": ac_answer_date,
            "Answer_upvote_count": ac_answer_upvote_count,
            "Answer_body": ac_answer_body,
            "Answer_comment_count": ac_answer_comment_count,
            "Answer_has_accepted": True
        })

    # other_answers
    answers_lst = driver.find_elements(
        By.XPATH, '//div[@class="widget widget-nopad answer-list"]/div[@class="widget-content"]/div[@class="post-container answer-container "]')
    # print("len:", len(answers_lst))

    for i in range(len(answers_lst)):
        answer = answers_lst[i]
        answer_date = answer.find_element(
            By.XPATH, './/time[@role="presentation"]').get_attribute("datetime")
        answer_upvote_count = answer.find_element(
            By.XPATH, './/div[@class="vote-widget"]/span/span[2]').text
        answer_upvote_count = convert2num(answer_upvote_count)
        answer_body = answer.find_element(
            By.XPATH, './/div[@class="answer-body"]').get_attribute('innerText').strip()
        answer_comment_count = answer.find_element(
            By.XPATH, './/span[@class="control-score-counter comment-count"]').get_attribute("innerText")
        answer_comment_count = convert2num(answer_comment_count)

        # print("answer_date:", answer_date)
        # print("answer_upvote:", answer_upvote_count)
        # print("anaswer_body:", answer_body)
        # print("answer_comment_count:", answer_comment_count)
        # print("answer_has_accepted:", False)

        total_dict["Answers"].append({
            "Answer_creation_date": answer_date,
            "Answer_upvote_count": answer_upvote_count,
            "Answer_body": answer_body,
            "Answer_comment_count": answer_comment_count,
            "Answer_has_accepted": False
        })

    return total_dict


def get_url(driver, url):
    driver.get(url)

    urls_lst = []
    urls_node_lst = driver.find_elements(By.XPATH, '//h2[@class="title"]/a')

    for urls_node in urls_node_lst:
        urls_lst.append(urls_node.get_attribute('href'))

    return urls_lst


if True:
    #options = webdriver.ChromeOptions()
    #options.add_argument('--ignore-certificate-errors')
    driver = uc.Chrome()#options=options)

    base_url = 'https://learn.microsoft.com/en-us/answers/topics/25447/azure-machine-learning.html?page='
    posts = []
    index = 0

    while True:
        index += 1
        print(index)
        page_url = base_url + str(index)
        posts_url = get_url(driver, page_url)

        if not posts_url:
            break

        for url in posts_url:
            posts.append(get_data(driver, url))

    posts_json = json.dumps(posts)
    with open(os.path.join('../Dataset/Raw', 'Azure Machine Learning.json'), 'w') as f:
        f.write(posts_json)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
